# Lab2: Set Covering with Genetic Algorithms
Here is the implementation of a basic genetic algotrithm for the set covering problem

In [23]:
import random
import logging

Here there are all tunable hyperparameters

In [24]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 40
NUM_GENERATIONS = 1000
MUTATION_RATE=0.7

In [25]:
def problem(N, seed=None):
    state = random.getstate()
    random.seed(seed)
    p = [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]
    random.setstate(state)
    return p


In [26]:
__CALLS__ = dict()


def CallCounter(fn):
    """Annotation @CallCounter"""
    assert fn.__name__ not in __CALLS__, f"Function '{fn.__name__}' already listed in __CALLS__"
    __CALLS__[fn.__name__] = 0
    logging.debug(f"CallCounter: Counting __CALLS__['{fn.__name__}'] ({fn})")

    def call_count(*args, **kwargs):
        __CALLS__[fn.__name__] += 1
        return fn(*args, **kwargs)

    return call_count

# Fitness and Tournament functions
The tournament function is used with the default tournament size of 2, select two random individuals and then pick the fittest.
The fitness function is the number of covered elements and then, in case of ex aequo, it compares the difference between N and the solution length, that is the distance from the optimal solution (called bloat at the end).

The first measure should actually be the condition of feasibility, i've put it as a soft measure instead of an hard check in order to allow also unfeasible solutions that may bring us to a good solution in future generations.
In reality putting this measure first gives same results as checking the validity of all individuals at each generations. 

The check on validity of a solutions is made only at the end, in case the algorithm doesn't find any solution, we could also eliminate the function.

In [27]:
def isvalid(solution):
    selected = set()
    for _ in solution:
        selected = selected | set(_)
    return selected==set(range(N))

@CallCounter
def fitness(state):
    solution = state[0]
    selected = set()
    for _ in solution:
        selected=selected | set(_)
    return len(selected),N-sum(len(_) for _ in solution)

def tournament(population,tournament_size=2):
    return max(random.choices(population,k=tournament_size), key=lambda i:i[2])

# Genetic Operators

The genetic operators used are really straightforward, the mutation swap a list in the solution with another left out list, picking from "available", tweaking a solution with a different list that may make it valid or reduce the number of repeated numbers.

The crossover operator select a cut point for each parent and then create the offspring selcting the first list from parent 1 and the rest from parent 2. This modify also the overall length of the solution, contrary to mutation.  

In [28]:
def mutation(state):
    solution,available,_=state

    removeIdx=random.randint(0,len(solution)-1) if len(solution)>0 else 0
    addIdx=random.randint(0,len(available)-1) if len(available)>0 else 0

    solution=solution[:removeIdx]+solution[removeIdx+1:]
    if len(available)>0:
        solution+=(available[addIdx],)
    available=available[:addIdx]+available[addIdx+1:]+(solution[removeIdx],)
    
    f=fitness((solution,available))
    return (solution,available,f)

def crossover(p1,p2):
    solution1, available, _ = p1
    solution2, _, _ = p2
    cut1=random.randint(0,len(solution1))
    cut2 = random.randint(0, len(solution2))

    solution = tuple(set((*solution1[: cut1], *solution2[cut2 :])))
    newAvailable=tuple((set(solution1)|set(available))-set(solution))
    f = fitness((solution, available))
    return (solution, newAvailable,f)

# Genetic Algorithm
The genetic algorithm initializes the population selecting random lists in each individual. Then at each generation it generates each offspring using either mutation or crossover. Offspring are added to the previous population. Unfeasible solutions are discarded only at the end.

In [29]:
def geneticAlgorithm():

    lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    #remove duplicates
    tuples = tuple(tuple(_) for _ in set(tuple(l) for l in lists))

    #generate initial population, random
    population = list()
    for genome in [tuple(random.choices(tuples,k=random.randint(1,len(tuples)))) for _ in range(POPULATION_SIZE)]:
        available=tuple(set(tuples)-set(genome))
        f=fitness((tuple(set(genome)), available))
        population.append((tuple(set(genome)),available,f))

    #population.append((tuples,tuple(),fitness((tuples,tuple())))) #tried to add all tuples as an individual

    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < MUTATION_RATE:
                p = tournament(population)
                o = mutation(p)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = crossover(p1, p2)
                
            offspring.append(o)
        population += offspring
        # sort and select the fittest mu
        population = sorted(population, key=lambda i: i[2], reverse=True)[:POPULATION_SIZE]
        if(population[2]==N):
            break
    population=tuple(_ for _ in population if isvalid(_[0]))
    solution=population[0][0]  #takes genome (solution) of the fittest valid individual

    print(f"Solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution) - N) / N * 100:.0f}%)")


In [30]:
for N in [5, 10, 20, 100,500,1000]:
    geneticAlgorithm()
    print(__CALLS__)

Solution for N=5: w=5 (bloat=0%)
{'fitness': 40100}
Solution for N=10: w=10 (bloat=0%)
{'fitness': 80200}
Solution for N=20: w=24 (bloat=20%)
{'fitness': 120300}
Solution for N=100: w=168 (bloat=68%)
{'fitness': 160400}
Solution for N=500: w=1388 (bloat=178%)
{'fitness': 200500}
Solution for N=1000: w=3330 (bloat=233%)
{'fitness': 240600}
